# House Prices Random Forest

This notebook will utilize some functions from the [fastai library](https://docs.fast.ai/) which aims to provide accessory functions to more easily execute tasks which are helpful to machine learning methods.

Data was obtained from Kaggle via the House Prices dataset.

**Goal:** Predict the sales price for each house.  

For each ID in the test set, predict the value of the SalePrice variable.  

Submissions are evaluated on **Root-Mean-Squared-Error (RMSE)** between the logarithm of the predicted value and the logarithm of the observed sales price. (Taking logs means that errors in predicting expensive houses and cheap houses will affect the result equally.)

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [3]:
# set a variable to where we put the kaggle data
PATH = "/home/ubuntu/housing_prices/data"

In [4]:
# reference the python variable "PATH" with unix command "ls" to make sure we are pointing to the right location
!ls {PATH}

data_description.txt  sample_submission.csv  test.csv  train.csv


In [5]:
# read in the training data with pandas
# from heading the training file, it appears there are no date columns
df_raw = pd.read_csv(f'{PATH}/train.csv', low_memory=False)

In [6]:
df_raw.shape

(1460, 81)

In [7]:
df_raw.tail().transpose()

,1455,1456,1457,1458,1459
Id,1456,1457,1458,1459,1460
MSSubClass,60,20,70,20,20
MSZoning,RL,RL,RL,RL,RL
LotFrontage,62,85,66,68,75
LotArea,7917,13175,9042,9717,9937
...,...,...,...,...,...
MoSold,8,2,5,4,6
YrSold,2007,2010,2010,2010,2008
SaleType,WD,WD,WD,WD,WD
SaleCondition,Normal,Normal,Normal,Normal,Normal


In [8]:
# define function to view the whole dataset
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max.columns", 1000):
        display(df)

In [9]:
display_all(df_raw.tail().T)

# potential feature engineering to change/add
#  difference between YearRemodAdd and YearBuilt? Use YrSold anywhere? 


,1455,1456,1457,1458,1459
Id,1456,1457,1458,1459,1460
MSSubClass,60,20,70,20,20
MSZoning,RL,RL,RL,RL,RL
LotFrontage,62,85,66,68,75
LotArea,7917,13175,9042,9717,9937
Street,Pave,Pave,Pave,Pave,Pave
Alley,NaN,NaN,NaN,NaN,NaN
LotShape,Reg,Reg,Reg,Reg,Reg
LandContour,Lvl,Lvl,Lvl,Lvl,Lvl
Utilities,AllPub,AllPub,AllPub,AllPub,AllPub


In [10]:
# replace the SalePrice column with its log
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [11]:
df_raw.tail().transpose()

,1455,1456,1457,1458,1459
Id,1456,1457,1458,1459,1460
MSSubClass,60,20,70,20,20
MSZoning,RL,RL,RL,RL,RL
LotFrontage,62,85,66,68,75
LotArea,7917,13175,9042,9717,9937
...,...,...,...,...,...
MoSold,8,2,5,4,6
YrSold,2007,2010,2010,2010,2008
SaleType,WD,WD,WD,WD,WD
SaleCondition,Normal,Normal,Normal,Normal,Normal


### Initial Processing

We need to process our data so that it will work with scikit learn's RandomForestRegressor.  

This includes:
* converting categorical string variables to "categories" (which encode the numeric information necessary for machine learning)
* performing feature extractions if there are dates for example
* reordering any ordinal variable categories to make more sense ("high", "medium", "low")
* taking care of any missing data, which we cannot pass directly to a Random Forest

The categorical variables are currently stored as strings, which is inefficient, and doesn't provide the numeric coding required for a random forest. Therefore we call the fastai function train_cats to convert strings to pandas categories.

In [12]:
df_raw.GarageType.dtype

dtype('O')

In [13]:
train_cats(df_raw)

In [14]:
# check various features that may be ordinal to see if the order makes sense
df_raw.Condition2.cat.categories

Index(['Artery', 'Feedr', 'Norm', 'PosA', 'PosN', 'RRAe', 'RRAn', 'RRNn'], dtype='object')

We now need to take care of any missing data.

In [15]:
# this code displays the percentage of each column that contains missing values
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

1stFlrSF         0.000000
2ndFlrSF         0.000000
3SsnPorch        0.000000
Alley            0.937671
BedroomAbvGr     0.000000
BldgType         0.000000
BsmtCond         0.025342
BsmtExposure     0.026027
BsmtFinSF1       0.000000
BsmtFinSF2       0.000000
BsmtFinType1     0.025342
BsmtFinType2     0.026027
BsmtFullBath     0.000000
BsmtHalfBath     0.000000
BsmtQual         0.025342
BsmtUnfSF        0.000000
CentralAir       0.000000
Condition1       0.000000
Condition2       0.000000
Electrical       0.000685
EnclosedPorch    0.000000
ExterCond        0.000000
ExterQual        0.000000
Exterior1st      0.000000
Exterior2nd      0.000000
Fence            0.807534
FireplaceQu      0.472603
Fireplaces       0.000000
Foundation       0.000000
FullBath         0.000000
Functional       0.000000
GarageArea       0.000000
GarageCars       0.000000
GarageCond       0.055479
GarageFinish     0.055479
GarageQual       0.055479
GarageType       0.055479
GarageYrBlt      0.055479
GrLivArea   

Many of the features have no missing data.

Some features have a lot of missing data, such as:
* Alley
* Fence
* FirePlaceQu
* MiscFeature
* PoolQC

Recall that pandas handles missing categorical variables by default, setting them to -1, so we only need to deal with missing continuous variables.

Let's see which of these columns/features may be continuous.

In [16]:
df_raw.Alley.unique()
df_raw.Fence.unique()
df_raw.FireplaceQu.unique()
df_raw.MiscFeature.unique()
df_raw.PoolQC.unique()

[NaN, Ex, Fa, Gd]
Categories (3, object): [Ex < Fa < Gd]

It looks like none of the worst offending columns are continuous data. The column with the most missing continupus data may be LotFrontage with ~17% missing.

To deal with the missing continous data, we can make use of the fastai function **proc_df()**, which:
* replaces missing continuous variables with the median, and also adds an additional column specifying which entries were replaced.
* turns the missing categorical variables to a value of 0 instead of -1 by adding 1 (if the variable is not numeric, then this function replaces it with its number codes (provided by cat.codes) and adds 1).
* removes the dependent variable from the df (and saves it as a new variable which we call when we fit the model).

In [17]:
# run proc_df() on our training data
df, y, nas = proc_df(df_raw, 'SalePrice')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na
0,1,60,4,65.0,8450,2,0,4,4,1,...,0,0,0,2,2008,9,5,False,False,False
1,2,20,4,80.0,9600,2,0,4,4,1,...,0,0,0,5,2007,9,5,False,False,False
2,3,60,4,68.0,11250,2,0,1,4,1,...,0,0,0,9,2008,9,5,False,False,False
3,4,70,4,60.0,9550,2,0,1,4,1,...,0,0,0,2,2006,9,1,False,False,False
4,5,60,4,84.0,14260,2,0,1,4,1,...,0,0,0,12,2008,9,5,False,False,False


We now have something we can pass to a random forest!

Use scikit-learn's RandomForestRegressor to create an instance and fit the model with the independent and dependent variables.

In [23]:
# We are predicting a continuous variable (Price), so we need a regressor
# create the instance of the random forest regressor
m = RandomForestRegressor(n_jobs=-1, random_state=1)

# "fit" the model
# really, this builds a forest of trees from the training set
m.fit(df, y)

# return the coefficient of determination from the prediction
# default prediction is self.predict(X, y)
m.score(df,y)

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9757027064236335

This value is the coefficient of determination, or the r^2 (r squared) value. It is the proportion of the variance in the dependent variable that is predictable from the independent variables.

0.97 is a very good r^2 value, **BUT** this is only on the training data. We are most likely overfitting and can expect a lower r^2 for the validation and test datasets.

Define a function to split up the training dataset into a validation dataset.

In [35]:
# make use of the ".copy()" method which creates a shallow copy of a list,
#   the original list remains unchanged
# supply an input df 'a' and an integer 'n' to split the df rows
# first, define the new training set as: "start at the beginning row and proceed to "n_trn"th row"
# next, define the validation set as: "begin at the "n_trn"th row and proceed to the last row"
def split_vals(a,n):
    return a[:n].copy(), a[n:].copy()

# define number of rows/samples for the validation set
n_valid = 365  # 25% of the total samples, same as default for "train_test_split" function from sklearn.model_selection
# calculate number of rows/samples for training set (nrow in original training set - nrow in validation set)
n_trn = len(df_raw) - n_valid

# perform "split_vals" on original raw training dataframe, specifying number of rows to keep in new training set
raw_train, raw_valid = split_vals(df_raw, n_trn)

# df_raw has 1460 rows
# we defined n_valid as 50, so n_trn = 1460-50=1410
# raw_train now has the first 1410 rows
# raw_valid has the last 50 rows

# Also run this same thing on our processed df (df, with SalePrice removed)
#  and on the "y" dependent variable "SalePrice"
# Both of these objects were created above from "proc_df" right before we ran the Random Forest
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

In [36]:
X_train.shape, y_train.shape, X_valid.shape

((1095, 83), (1095,), (365, 83))

Let's try our model again, this time with separate training and validation sets.

In [37]:
# define function to calculate Root Mean Squared Error (RMSE)
# take x - y, square it, take the mean of all those, take the square root of this mean
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

# define a "print_score" function to calculate and print the:
#  RMSE (defined above) of the random forest for the training set and validation set predictions
#  score (r^2) of the random forest for the training set and validation set
def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [38]:
# fit a random forest regressor on the new training set and y set (1410 'samples', depending on validation set size)
m = RandomForestRegressor(n_jobs=-1, random_state=1) # create the instance, use all cores
%time m.fit(X_train, y_train) # build the forest of trees from the training data, report the time
print_score(m) # execute the "print_score" function which fits the model on training and validation datasets and reportd the RMSE and score (r^2) for both


/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 96 ms, sys: 0 ns, total: 96 ms
Wall time: 113 ms
[0.11534417987892508, 0.15900357921754776, 0.9186107889506997, 0.8284810845893918]


The r^2 of the validation set is ~0.83, a significant drop from the ~0.97 r^2 of the training set. This suggests that we are overfitting badly.

Recall that a random forest is a collection of many single decision trees.

This concept of averaging methods is called **ensembling** and is an important concept.

By default, random forest in scikit learn subsamples all of the data with replacement. This is called **bootstrapping**. Example: if you have 10,000 rows, it will 'subsample' 10,000 rows by randomly making 10,000 selections, with replacement. This typically results in 63.2% of the rows being represented (and a bunch of them will be represented multiple times).

Random forests have a very clever trick called **out-of-bag (OOB) error**. The idea is to calculate error on the training set, but only include the trees in the calculation of a row's error where that row was not included in training that tree. This allows us to see whether the model is over-fitting, without needing a separate validation set. This also has the benefit of allowing us to see whether our model generalizes, even if we only have a small amount of data so want to avoid separating some out to create a validation set.

OOB score can be used in a grid search (from scikit learn) to optimize hyperparameters. Choose the combo with the best OOB score.

To get the OOB score of our random forest, we will add one more parameter to our model constructor code.

In [39]:
m = RandomForestRegressor(n_jobs=-1, random_state=1, oob_score=True)
%time m.fit(X_train, y_train)
print_score(m)

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 96 ms, sys: 8 ms, total: 104 ms
Wall time: 120 ms
[0.11534417987892513, 0.15900357921754774, 0.9186107889506998, 0.8284810845893921, 0.8326200857220672]


We see that our OOB score is 0.83. Let's interpret the OOB score. 

The OOB score is the score of the out-of-bag samples.

Scikit Learn uses 'score' to mean "measure of how good a model is" which is different for different models. For a RandomForestRegressor, this is the r^2 (coefficient of determination). This is defined by:

(1 - u/v)

where u is the regression's sum squared error and v is the sum squared error of the best constant predictor.

So the OOB score (r^2) can be negative if u > v, i.e. the model is worse than the best constant predictor. This basically means the model is not good, usually models get positive scores.

### Subsampling

One of the easiest ways to avoid over-fitting is also one of the best ways to speed up analysis: **subsampling**

The basic idea is this: rather than limit the total amount of data that our model can access, let's instead limit it to a different random subset per tree. That way, given enough trees, the model can still see all the data, but for each individual tree it'll be just as fast as if we had cut down our dataset as before.

In [45]:
# use this fastai function to give each tree a random sample of n random rows
# default is to use all rows with replacement for random subset size
set_rf_samples(1000)

# Note: can also reset this with 'reset_rf_samples()'

In [46]:
m = RandomForestRegressor(n_jobs=-1, random_state=1, oob_score=True)
%time m.fit(X_train, y_train)
print_score(m)

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


CPU times: user 180 ms, sys: 4 ms, total: 184 ms
Wall time: 119 ms
[0.06781318500045705, 0.15751957135311492, 0.9718677963457786, 0.8316677752173856, -3.0458335976940525]


In [47]:
set_rf_samples(800)
m = RandomForestRegressor(n_jobs=-1, random_state=1, oob_score=True)
%time m.fit(X_train, y_train)
print_score(m)

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 156 ms, sys: 0 ns, total: 156 ms
Wall time: 119 ms
[0.08131860239276344, 0.1626105387036635, 0.9595465766442438, 0.8206110871732095, 0.8249569823098848]


In [53]:
set_rf_samples(500)
m = RandomForestRegressor(n_jobs=-1, random_state=1, oob_score=True)
%time m.fit(X_train, y_train)
print_score(m)

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 120 ms, sys: 0 ns, total: 120 ms
Wall time: 120 ms
[0.0999631587959114, 0.15839630824345904, 0.938869857224372, 0.8297887225291076, 0.8390125446528369]


Note the improvement of the OOB score with varying (decreasing) number of samples being set.

Another way to reduce over-fitting is to grow our trees less deeply. We do this by specifying (with **min_samples_leaf** parameter of RandomForestRegression) that we require some minimum number of samples in every leaf node. This has two benefits:

1. There are less decision rules for each leaf node; simpler models should generalize better
2. The predictions are made by averaging more rows in the leaf node, resulting in less volatility

In [63]:
m = RandomForestRegressor(n_jobs=-1, random_state=1, min_samples_leaf=3, oob_score=True)
%time m.fit(X_train, y_train)
print_score(m)

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 88 ms, sys: 4 ms, total: 92 ms
Wall time: 119 ms
[0.11245191589186028, 0.15705166817453772, 0.9226412958982962, 0.8326663330666318, 0.8388649220813118]


We can also increase the amount of variation amongst the trees by not only using a sample of rows for each tree, but by also using a sample of columns for each split. We do this by specifying **max_features** parameter, which is the proportion of features to randomly select from at each split.

In [72]:
m = RandomForestRegressor(n_jobs=-1, random_state=1, min_samples_leaf=3, max_features=0.75, oob_score=True)
%time m.fit(X_train, y_train)
print_score(m)

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 76 ms, sys: 0 ns, total: 76 ms
Wall time: 119 ms
[0.11190825091491619, 0.1529953853713687, 0.9233874913172228, 0.8411983961048176, 0.8414031567876514]


We have reached a validation set score of 0.84. This may be able to be further improved with other techniques, such as feature engineering.